In [ ]:
# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).
# Source for "Build a Large Language Model From Scratch"
#   - https://www.manning.com/books/build-a-large-language-model-from-scratch
# Code: https://github.com/rasbt/LLMs-from-scratch

# 6장의 주요 내용을 요약한 파일: 분류(Classification)를 위한 GPT 미세 조정
import requests
import zipfile
import os
from pathlib import Path
import time

import matplotlib.pyplot as plt
import pandas as pd
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

# 이전 챕터나 별도 모듈에서 정의된 GPT 모델 관련 함수들 임포트
from previous_chapters import GPTModel, load_gpt2_model

# -----------------------------------------------------------------------------
# 1. 데이터 준비 유틸리티 함수들
# -----------------------------------------------------------------------------

def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    """
    스팸 데이터셋(SMS Spam Collection)을 다운로드하고 압축을 해제하는 함수
    """
    if data_file_path.exist():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
    return

    # 파일 다운로드 (스트리밍 방식)
    response=  requests.get(url, stream=True, timeout=60)
    response.raise_for_status()
    with open(zip_path, "wb") as out_file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                out_file.write(chunk)
     
    # 압축 해제
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)
        
    # 압축 해제된 파일에 .tsv 확장자 추가 (Pandas로 읽기 편하게)
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as "{data_file_path}")

def create_balanced_dataset(df):
    """
    데이터 불균형 해결을 위한 함수.
    스팸(spam) 데이터 수에 맞춰 햄(ham, 정상 메일) 데이터를 언더샘플링합니다.
    """
    # "spam" 라벨의 개수 계산
    num_spam = df[df["Label"] == "spam"].shape[0]
          
    # "ham" 데이터 중에서 "spam" 개수만큼만 무작위 추출
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    
    # 두 데이터셋 병합
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
          
    return balanced_df

def random_split(df, train_frac, validation_frac):
    """
    데이터셋을 학습(Train), 검증(Validation), 테스트(Test) 셋으로 분할하는 함수
    """
    # 전체 데이터 섞기
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
          
    # 분할 지점(인덱스) 계산
    train_end = int(len(df)*train_frac)
    validataion_end = train_end + int(len(df)*validation_frac)
          
    #데이터 분할
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    
    return train_df, validation_df, test_df

class SpamDataset(Dataset):
    """
    PyTorch Dataset 클래스 정의.
    텍스트를 토큰화하고, 패딩(Padding) 처리를 수행합니다.
    """
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        
        # 1. 텍스트 데이터를 토큰 ID 리스트로 변환 (Tokenization)
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]
        
        # 2. 최대 길이(max_length) 설정
        if max_length is None:
            self.max_length = self._longest_encoded_length() 
          
        
        
        
          
          
          
